In [1]:
import pandas as pd
import matrix_profile as mp
import numpy as np
from matrix_profile import core

In [2]:
df = pd.read_table("https://raw.githubusercontent.com/stanford-futuredata/ASAP/master/Taxi.csv", sep=',')
df['value'] = df['value'].astype(np.float64)

In [3]:
m = 48

In [4]:
stomp_out = mp.stomp.stomp(df['value'].values, df['value'].values, m=m, ignore_trivial=True)

In [5]:
out_df = pd.DataFrame(stomp_out, columns=['mp', 'inx', 'left', 'right'])

In [6]:
# Plot stuff